In [1]:
import gensim as gs
import nltk
import pandas as pd
import numpy as np
import spacy as spc

In [2]:
%matplotlib inline

# Chequeo que exista el Directorio

In [3]:
!ls

DataExplorationAndTransformation.ipynb
DataTypes_POSTag.ipynb
dt_info_pagadores_muestra
dt_trxpse_personas_2016_2018_muestra_adjt
fasttext-sbwc.3.6.e20.vec


In [6]:
data = pd.read_csv('dt_trxpse_personas_2016_2018_muestra_adjt/dt_trxpse_personas_2016_2018_muestra_adjt_V1.csv', header = None, quoting = 3)

In [7]:
header = ["id_trn_ach",
"id_cliente",
"fecha",
"hora",
"valor_trx",
"ref1",
"ref2",
"ref3",
"sector",
"subsector",
"descripcion"]

In [8]:
data.columns = header

In [9]:
data.head(20)

,id_trn_ach,id_cliente,fecha,hora,valor_trx,ref1,ref2,ref3,sector,subsector,descripcion
0,230435642,3,20161207.0,113451,2122392.51,CC,NaN,NaN,\N,\N,\N
1,222356110,10,20161016.0,3424,148438.37,Referencia: Contrato: Valor:,CC,NaN,\N,\N,\N
2,309137749,10,20180120.0,195042,94025.19,CC,NaN,NaN,\N,\N,\N
3,324614737,10,20180326.0,192146,94430.07000000001,CC,NaN,NaN,\N,\N,\N
4,235344690,18,20170106.0,201317,670645.5699999999,MEDICINA PREPAGADA COLSANITAS,CE,NaN,\N,\N,\N
5,320049316,18,20180307.0,143513,706933.7,RECAUDO COLSANITAS,CC,NaN,\N,\N,\N
6,230519178,19,20161207.0,155840,306773.79,CC,NaN,NaN,\N,\N,\N
7,241307506,19,20170210.0,142809,701067.98,Pago de la Planilla Cesantias,CEDULA DE CIUDADANIA,NaN,SERVICIOS FINANCIEROS,OTROS SERVICIOS FINANCIEROS,Otras actividades auxiliares de las actividade...
8,316193109,24,20180220.0,153148,767571.01,CC,NaN,NaN,\N,\N,\N
9,282076350,26,20170914.0,70026,100277.45,Pago de la factura # CONJUNTO RESIDENCIAL PIET...,IDC,NaN,\N,\N,\N


# Chequeo los largos de los ref1

In [10]:
maxlen = 0
for i,row in enumerate(data["ref1"]):
    try:
        largo = len(row.split())
        if largo > 1000:
            pass#print(row)
    except:
        largo = maxlen
        
    maxlen = max(maxlen,largo)

In [11]:
maxlen

20

# Esperable que sean maximo 20 caracteres no separados.

# Conversion a un solo formato de fecha

In [12]:
data.head()

,id_trn_ach,id_cliente,fecha,hora,valor_trx,ref1,ref2,ref3,sector,subsector,descripcion
0,230435642,3,20161207.0,113451,2122392.51,CC,NaN,NaN,\N,\N,\N
1,222356110,10,20161016.0,3424,148438.37,Referencia: Contrato: Valor:,CC,NaN,\N,\N,\N
2,309137749,10,20180120.0,195042,94025.19,CC,NaN,NaN,\N,\N,\N
3,324614737,10,20180326.0,192146,94430.07000000001,CC,NaN,NaN,\N,\N,\N
4,235344690,18,20170106.0,201317,670645.5699999999,MEDICINA PREPAGADA COLSANITAS,CE,NaN,\N,\N,\N


In [15]:
data['fecha_string'] = data['fecha'].astype(int).astype(str)
data['hora_string'] = data['hora'].astype(str)

In [20]:
data['hora_string_final'] = data['hora_string'].apply(lambda x: x.zfill(6))
data['fecha_string_final'] = data['fecha_string'].apply(lambda x: x.split('.')[0])

# Se puede observar que hay nan y /n, probablemente de un mal parsing. Se reemplaza esto por none.

In [25]:
data['hora_string_final'].value_counts()

0000\N    12127
000nan      597
094501      344
110337      342
104327      339
112201      334
110020      334
111525      331
105835      331
112241      331
115034      331
112722      329
103142      328
111440      328
104501      327
112249      327
105821      325
105901      325
111659      325
105855      325
110055      325
110531      325
105431      325
153001      324
112151      324
114357      324
111126      323
114328      323
114821      323
112123      322
          ...  
032323        1
025635        1
035128        1
031333        1
034617        1
024536        1
025703        1
022453        1
025154        1
032855        1
032439        1
031852        1
031728        1
030119        1
024551        1
035037        1
040226        1
013952        1
024335        1
034520        1
035747        1
034909        1
034850        1
040948        1
025029        1
023349        1
033012        1
032342        1
035742        1
031539        1
Name: hora_string_final,

In [35]:
data['hora_string_final_clean'] = data['hora_string_final'].apply(lambda x: np.nan if x =="0000\\N" else x).apply(lambda x: np.nan if x =="000nan" else x)

In [40]:
data['fecha_usable'] = pd.to_datetime(data['fecha_string_final'] + ' ' + data['hora_string_final_clean'], format="%Y%m%d %H%M%S")

In [43]:
data.dtypes

id_trn_ach                         object
id_cliente                         object
fecha                             float64
hora                               object
valor_trx                          object
ref1                               object
ref2                               object
ref3                              float64
sector                             object
subsector                          object
descripcion                        object
fecha_string                       object
hora_string                        object
hora_string_pad                    object
hora_string_final                  object
fecha_string_final                 object
hora_string_final_clean            object
fecha_usable               datetime64[ns]
dtype: object

# Procedemos a eliminar las columnas ruido del dataframe

In [46]:
data_date_clean = data.drop(['fecha_string','hora_string','hora_string_pad','hora_string_final','fecha_string_final','hora_string_final_clean', 'fecha', 'hora'], axis=1)

# Se puede ver que tambien se necesitan otras conversiones para que todo sea coherente.

In [47]:
data_date_clean.head()

,id_trn_ach,id_cliente,valor_trx,ref1,ref2,ref3,sector,subsector,descripcion,fecha_usable
0,230435642,3,2122392.51,CC,NaN,NaN,\N,\N,\N,2016-12-07 11:34:51
1,222356110,10,148438.37,Referencia: Contrato: Valor:,CC,NaN,\N,\N,\N,2016-10-16 00:34:24
2,309137749,10,94025.19,CC,NaN,NaN,\N,\N,\N,2018-01-20 19:50:42
3,324614737,10,94430.07000000001,CC,NaN,NaN,\N,\N,\N,2018-03-26 19:21:46
4,235344690,18,670645.5699999999,MEDICINA PREPAGADA COLSANITAS,CE,NaN,\N,\N,\N,2017-01-06 20:13:17


In [50]:
data_date_clean.dtypes

id_trn_ach              object
id_cliente              object
valor_trx               object
ref1                    object
ref2                    object
ref3                   float64
sector                  object
subsector               object
descripcion             object
fecha_usable    datetime64[ns]
dtype: object

In [51]:
data_date_clean['valor_trx'] = pd.to_numeric(data_date_clean['valor_trx'])

ValueError: Unable to parse string "\N" at position 1238

Como podemos ver no se puede convertir automaticamente, por lo que ponemos un coerce.

In [52]:
data_date_clean['valor_trx'] = pd.to_numeric(data_date_clean['valor_trx'], errors='coerce')

In [53]:
data_date_clean.dtypes

id_trn_ach              object
id_cliente              object
valor_trx              float64
ref1                    object
ref2                    object
ref3                   float64
sector                  object
subsector               object
descripcion             object
fecha_usable    datetime64[ns]
dtype: object

# Notamos tambien que el /N pareciera ser el NaN de esta data. Antes de cerrar la etapa de calidad de datos por tipos e inconsistencias tecnologicas analizaremos un poco mas este /N

In [66]:
condicion_interesante = data_date_clean == '\\N'

In [67]:
condicion_interesante.sum()

id_trn_ach            0
id_cliente            0
valor_trx             0
ref1              12127
ref2                  0
ref3                  0
sector          8534270
subsector       8534270
descripcion     8534270
fecha_usable          0
dtype: int64

In [68]:
data_date_clean_no_n = data_date_clean
data_date_clean_no_n[condicion_interesante] = np.nan

In [70]:
condicion_interesante_check = data_date_clean_no_n == '\\N'

In [71]:
condicion_interesante_check.sum()

id_trn_ach      0
id_cliente      0
valor_trx       0
ref1            0
ref2            0
ref3            0
sector          0
subsector       0
descripcion     0
fecha_usable    0
dtype: int64

# Se puede notar que no hay ningun N que altere. Por lo que procedemos a cerrar la calidad de datos

In [72]:
data_date_clean_no_n.head()

,id_trn_ach,id_cliente,valor_trx,ref1,ref2,ref3,sector,subsector,descripcion,fecha_usable
0,230435642,3,2122392.51,CC,NaN,NaN,NaN,NaN,NaN,2016-12-07 11:34:51
1,222356110,10,148438.37,Referencia: Contrato: Valor:,CC,NaN,NaN,NaN,NaN,2016-10-16 00:34:24
2,309137749,10,94025.19,CC,NaN,NaN,NaN,NaN,NaN,2018-01-20 19:50:42
3,324614737,10,94430.07,CC,NaN,NaN,NaN,NaN,NaN,2018-03-26 19:21:46
4,235344690,18,670645.57,MEDICINA PREPAGADA COLSANITAS,CE,NaN,NaN,NaN,NaN,2017-01-06 20:13:17


# Con esto cerramos la transformacion de fecha y de Problemas de N.
# El proximo Notebook, se enfocara en la estandarizacion de los campos libres y outlier detection.

In [74]:
data_date_clean_no_n.to_pickle('dt_trxpse_personas_2016_2018_muestra_adjt/dt_trxpse_V2.pkl.gzip')